This example was adapted from https://www.cs.jhu.edu/~mdredze/datasets/image_spam/

In [1]:
from pyBWMD import vectorize
import numpy as np
from multiprocessing import Pool
import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import urllib.parse
import glob

import xgboost as xgb

import matplotlib.pyplot as plt

/Users/eman7613/opt/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /Users/eman7613/opt/anaconda3/lib/python3.7/site-packages/pyBWMD-0.0.1-py3.7.egg/pyBWMD/suffixArray.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [2]:
spam_url = "https://www.cs.jhu.edu/~mdredze/datasets/image_spam/personal_image_spam.tar.gz"
ham_url = "https://www.cs.jhu.edu/~mdredze/datasets/image_spam/personal_image_ham.tar.gz"

In [3]:
if not os.path.exists("personal_image_spam"):
    print("Downloading dataset")
    import urllib.request
    import tarfile

    urllib.request.urlretrieve(ham_url, 'personal_image_ham.tar.gz')
    urllib.request.urlretrieve(spam_url, 'personal_image_spam.tar.gz')
    
    tf = tarfile.open("personal_image_ham.tar.gz")
    tf.extractall()
    tf = tarfile.open("personal_image_spam.tar.gz")
    tf.extractall()
    

In [4]:
spam_paths = glob.glob("personal_image_spam/*")
ham_paths = glob.glob("personal_image_ham/*")


all_paths = spam_paths + ham_paths
yBad = [1 for i in range(len(spam_paths))]
yGood = [0 for i in range(len(ham_paths))]
y = yBad + yGood


In [5]:
X = vectorize(all_paths)



In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #splitting data


In [9]:
lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train) #training our model


/Users/eman7613/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
##############
# Evaluation #
##############

predicted = lgs.predict(X_test)

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

Accuracy: 0.935150
Precision: 0.939130
Recall: 0.960000
F1-Score: 0.949451
AUC: 0.968577
